In [1]:
from time_stitches import *
from step1_functions import *
from features import get_jerk
from functions import *
import os
import warnings
warnings.filterwarnings('ignore')
from step1_functions import smooth
from scipy.signal import butter, filtfilt
import pycircstat

In [2]:
def get_path_length_g(time, pos):

    dt = 1/120
    diff_time = np.diff(time, axis=0)
    mask_step_bigger_dt = (diff_time>dt)

    pos_selected = pos[1:][~mask_step_bigger_dt]

    # Calculate the differences in positions between consecutive samples
    differences = np.diff(pos_selected, axis=0)
    # Calculate the Euclidean distance (path length) between consecutive points
    distances = np.linalg.norm(differences, axis=1)
    # Sum up the distances to get the total path length
    total_path_length = np.sum(distances)
    return total_path_length

In [3]:
def compute_v_acc_jerk(data_one_stitch): #data including time and position
    dt = 1/120
    diff_t_x_y_z = np.diff(data_one_stitch, axis = 0)
    mask_step_bigger_dt = (diff_t_x_y_z[:,0]>dt)
    t_x_y_z = diff_t_x_y_z[~mask_step_bigger_dt]
    v_x_y_z = [t_x_y_z[:,1]/t_x_y_z[:,0], t_x_y_z[:,2]/t_x_y_z[:,0], t_x_y_z[:,3]/t_x_y_z[:,0]]
    acc_x_y_z = [v_x_y_z[0]/t_x_y_z[:,0], v_x_y_z[1]/t_x_y_z[:,0], v_x_y_z[2]/t_x_y_z[:,0]]
    jerk_x_x_z = [acc_x_y_z[0]/t_x_y_z[:,0], acc_x_y_z[1]/t_x_y_z[:,0], acc_x_y_z[2]/t_x_y_z[:,0]]
    return v_x_y_z, acc_x_y_z, jerk_x_x_z

In [4]:
def get_mean_std_v(v):
    return np.mean(np.linalg.norm(v, axis=0)), np.std(np.linalg.norm(v, axis=0))

In [5]:
def get_list_idle_time(subject, i, list_np_segmented_nh_rec, list_np_segmented_tw_rec):
    needle_holder_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
    tweezers_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')
    dict_segment_time = full_segments_time[i]
    
    full_time_segment_tw = pd_2_numpy_and_segment(tweezers_rec, dict_segment_time)
    full_time_segment_nh = pd_2_numpy_and_segment(needle_holder_rec, dict_segment_time)

    list_idle_nh = []
    list_idle_tw = []
    for s in range(8):
        effective_t_nh = list_np_segmented_nh_rec[s][:,0]
        effective_t_tw = list_np_segmented_tw_rec[s][:,0]
        tot_time_nh = full_time_segment_nh[s][:,0]
        tot_time_tw = full_time_segment_tw[s][:,0]
        idle_time_nh = tot_time_nh.shape[0] * (1.0 / 120.0) - effective_t_nh.shape[0] * (1.0 / 120.0)
        idle_time_tw = tot_time_tw.shape[0] * (1.0 / 120.0) - effective_t_tw.shape[0] * (1.0 / 120.0)
        list_idle_nh.append(idle_time_nh)
        list_idle_tw.append(idle_time_tw)
    return list_idle_nh, list_idle_tw


In [6]:
def get_economy_of_volume(segment_pos, PL):
    x = segment_pos[:,0]
    y = segment_pos[:,1]
    z = segment_pos[:,2]

    EOV = ((np.max(x)-np.min(x))*(np.max(y)-np.min(y))*(np.max(z)-np.min(z)))**(1/3)/PL
    return EOV

In [7]:
def get_mean_std_curvature (v, acc):
    v = np.array(v)
    acc = np.array(acc)
    
    with np.errstate(divide='ignore', invalid='ignore'):
        curvature = np.where(v == 0, np.nan, (v * acc) / (v**3))
    return np.nanmean(curvature), np.nanstd(curvature)

In [140]:
def inverse_quaternion(quaternion):
    q1, q2, q3, q4 = quaternion
    inverse = [q1, -q2, -q3, -q4]/(q1**2 + q2**2 +q3**2 +q4**2) #nécessaire si normalisé???
    return inverse

In [141]:
def get_orientation_change(quaternion_nh, quaternion_tw): #SUPPRIMER UNE FOIS QUE L'AUTRE FONCTIONNE
    #compute unitary quaternion
    norm_quaternion_nh =  quaternion_nh/np.linalg.norm(quaternion_nh, axis=0)
    norm_quaternion_tw =  quaternion_tw/np.linalg.norm(quaternion_tw, axis=0)

    #compute inverse (according to wikipedia formula)
    inverse_nh = []
    for i in range (len(norm_quaternion_nh)):
        inverse_nh.append(inverse_quaternion(norm_quaternion_nh[i,:]))
    inverse_tw = []
    for n in range (len(norm_quaternion_tw)):
        inverse_tw.append(inverse_quaternion(norm_quaternion_tw[n,:]))

    #compute rotation difference
    deltaQ_nh = []
    for j in range (len(norm_quaternion_nh)-1):
        q = np.array(norm_quaternion_nh[j])
        qinv = np.array(inverse_nh[j+1])
        deltaQ_nh.append((q*qinv).tolist())
    
    deltaQ_tw = []
    for j in range (len(norm_quaternion_tw)-1):
        q = np.array(norm_quaternion_tw[j])
        qinv = np.array(inverse_tw[j+1])
        deltaQ_tw.append((q*qinv).tolist())

    #compute orientation change
    deltaTeta_nh = []
    for p in range (len(deltaQ_nh)):
        delta_orientation = 2*np.cos(deltaQ_nh[p][0])**(-1)
        deltaTeta_nh.append(delta_orientation)

    deltaTeta_tw = []
    for p in range (len(deltaQ_tw)):
        delta_orientation = 2*np.cos(deltaQ_tw[p][0])**(-1)
        deltaTeta_tw.append(delta_orientation)
    
    return deltaTeta_nh, deltaTeta_tw
    

In [142]:
def get_orientation_change(quaternion):
    #compute unitary quaternion
    norm_quaternion =  quaternion/np.linalg.norm(quaternion, axis=0)
    
    #compute inverse (according to wikipedia formula)
    inverse = []
    for i in range (len(norm_quaternion)):
        inverse.append(inverse_quaternion(norm_quaternion[i,:]))

    #compute rotation difference
    deltaQ = []
    for j in range (len(norm_quaternion)-1):
        q = np.array(norm_quaternion[j])
        qinv = np.array(inverse[j+1])
        deltaQ.append((q*qinv).tolist())

    #compute orientation change
    deltaTeta = []
    for p in range (len(deltaQ)):
        delta_orientation = 2*np.cos(deltaQ[p][0])**(-1)
        deltaTeta.append(delta_orientation)
    
    return deltaTeta
    

In [143]:
def get_angular_displacement(quaternion):
    deltaTeta = get_orientation_change(quaternion)
    angular_disp = np.sum(np.abs(deltaTeta))
    return angular_disp

In [144]:
def get_rate_of_orientation_change(list_np_segmented):
    time = list_np_segmented[:,0]
    quaternion = list_np_segmented[:, 4:8]
    dt = 1/120
    diff_t = np.diff(time, axis = 0)
    mask_step_bigger_dt = (diff_t>dt)
    selected_time = time[1:][~mask_step_bigger_dt]
    selected_quaternion = quaternion[1:][~mask_step_bigger_dt]

    diff_selected_time = np.diff(selected_time, axis = 0)
    deltaTeta = get_orientation_change(selected_quaternion)
    rate_orientation_change = np.mean(np.abs(deltaTeta)/diff_selected_time)

    return rate_orientation_change

In [7]:
def compute_mask_both_tools(subject, i, segment_nh, segment_tw):
    dict_segment_time = full_segments_time[i]
    needle_holder_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
    tweezers_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')
    ref_nh_time = pd_2_numpy_and_segment(needle_holder_rec, dict_segment_time)[0][0,0]
    ref_tw_time = pd_2_numpy_and_segment(tweezers_rec, dict_segment_time)[0][0,0]

    delta_t = ref_nh_time - ref_tw_time
    time_nh = segment_nh[:,0]
    time_tw = segment_tw[:,0]

    mask_time_nh = np.zeros(len(time_nh))
    for index, t in enumerate(time_nh):
        diff = np.abs(t - np.array(time_tw)) <= (delta_t)
        if (np.sum(diff)>0): mask_time_nh[index]=1

    mask_time_tw = np.zeros(len(time_tw))
    for index, t in enumerate(time_tw):
        diff = np.abs(np.array(time_nh) - t) <= delta_t
        if (np.sum(diff)>0): mask_time_tw[index]=1
    
    return mask_time_nh.astype(int), mask_time_tw.astype(int)

In [8]:
def get_bimanual_dexterity3(subject, i, segment_nh, segment_tw):

    #v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(segment_nh)
    #v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(segment_tw)
    
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    
    if ((np.sum(mask_nh)<=10) or np.sum(mask_tw)<=10):
        BD=np.nan
    else: 
        nh_points = segment_nh[mask_nh.astype(bool)]
        tw_points = segment_tw[mask_tw.astype(bool)]
        v_nh, acc_nh, jerk_nh= compute_v_acc_jerk(nh_points)
        v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(tw_points)
        
        v_stitch_nh = np.linalg.norm(v_nh, axis=0)
        v_stitch_tw = np.linalg.norm(v_tw, axis=0)
            
        mean_v_tw = np.nanmean(v_stitch_tw)
        mean_v_nh = np.nanmean(v_stitch_nh)

        if (len(v_stitch_nh)==len(v_stitch_tw)):
            BD = np.sum((v_stitch_nh - mean_v_nh)*(v_stitch_tw - mean_v_tw))/np.sqrt(np.sum((v_stitch_nh - mean_v_nh)**2)*np.sum((v_stitch_tw - mean_v_tw)**2))
        else: 
            BD=np.nan
            print(f'For Subject {subject}, the 2 tools velocities do not have the same length')
        return BD
    #return difft_nh, difft_tw#BD

        

In [9]:
def get_bimanual_efficiency3(subject, i, segment_nh, segment_tw): #consider only time of selected moments
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    dt = 1/120
    time_nh = segment_nh[mask_nh.astype(bool)][:,0]
    #time_nh = segment_nh[:,0][mask_nh.astype(bool)] #since we have a list we need a boolean mask
    diff_nh = np.diff(time_nh)
    #we don't want to consider the difference of time between 2 points separated by an unselected period
    mask_nh = diff_nh>dt
    diff_nh_ = diff_nh[~mask_nh]
    sum_time_nh = np.sum(diff_nh_)

    time_whole_segment_nh = segment_nh[:,0]
    diff_whole_segment_nh = np.diff(time_whole_segment_nh)
    mask_whole = diff_whole_segment_nh>dt
    diff_whole = diff_whole_segment_nh[~mask_whole]
    sum_time_whole = np.sum(diff_whole)

    if (sum_time_nh ==0): #it is due to imprecision in the algorithm, we should in theory not have 0 values
        BE=np.nan
    else: 
        BE= sum_time_nh/sum_time_whole #sum time is the same for nh and tw

    return BE

In [10]:
def butter_lowpass_filter(data, cutoff=12, fs=120, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

In [11]:
def get_relative_phase(subject, i, segment_nh, segment_tw):
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    
    if ((np.sum(mask_nh)<=10) or np.sum(mask_tw)<=10):
        relative_phase_x=np.nan
        relative_phase_y=np.nan
        return relative_phase_x, relative_phase_y
    else: 
        nh_points = segment_nh[mask_nh.astype(bool)]
        tw_points = segment_tw[mask_tw.astype(bool)]

        dt = 1/120
        diff_t_x_y_z_nh = np.diff(nh_points, axis = 0)
        mask_step_bigger_dt_nh = (diff_t_x_y_z_nh[:,0]>dt)
        t_x_y_z_nh = diff_t_x_y_z_nh[~mask_step_bigger_dt_nh]
        v_x_y_z_nh = [t_x_y_z_nh[:,1]/t_x_y_z_nh[:,0], t_x_y_z_nh[:,2]/t_x_y_z_nh[:,0], t_x_y_z_nh[:,3]/t_x_y_z_nh[:,0]]
        x_y_z_nh = nh_points[1:,1:3][~mask_step_bigger_dt_nh]

        diff_t_x_y_z_tw = np.diff(tw_points, axis = 0)
        mask_step_bigger_dt_tw = (diff_t_x_y_z_tw[:,0]>dt)
        t_x_y_z_tw = diff_t_x_y_z_tw[~mask_step_bigger_dt_tw]
        v_x_y_z_tw = [t_x_y_z_tw[:,1]/t_x_y_z_tw[:,0], t_x_y_z_tw[:,2]/t_x_y_z_tw[:,0], t_x_y_z_tw[:,3]/t_x_y_z_tw[:,0]]
        x_y_z_tw = tw_points[1:,1:3][~mask_step_bigger_dt_tw]

        #print(v_x_y_z_nh)
        phi_nh_x = np.arctan(v_x_y_z_nh[0]/x_y_z_nh[:,0])
        phi_tw_x = np.arctan(v_x_y_z_tw[0]/x_y_z_tw[:,0])

        relative_phase_x = phi_nh_x - phi_tw_x

        phi_nh_y = np.arctan(v_x_y_z_nh[1]/x_y_z_nh[:,1])
        phi_tw_y = np.arctan(v_x_y_z_tw[1]/x_y_z_tw[:,1])

        relative_phase_y = phi_nh_y - phi_tw_y

        return relative_phase_x, relative_phase_y


In [12]:
from scipy.stats import circstd, circmean
def compute_circular_stat2(data):
    data_0_2pi = data+np.pi
    mean_rad = circmean(data_0_2pi, high = 2*np.pi, low=0, nan_policy='omit')
    std_rad = circstd(data_0_2pi, high = 2*np.pi, low=0, nan_policy='omit')
    #mean_rad = circmean(data, high = np.pi, low=-np.pi, nan_policy='omit')
    #std_rad = circstd(data, high = np.pi, low=-np.pi, nan_policy='omit')
    return np.rad2deg(mean_rad), np.rad2deg(std_rad)

In [41]:
def compute_circular_stat(data):
    mean_rad = pycircstat.mean(data)
    std_rad = pycircstat.std(data)
    print(mean_rad, std_rad)
    return mean_rad*180/np.pi, std_rad*180/np.pi

In [13]:
def get_relative_phase_mean_std(subject, i, segment_nh, segment_tw):
    relative_phase_x, relative_phase_y = get_relative_phase(subject, i, segment_nh, segment_tw)
    if ((np.isnan(relative_phase_x)).all() and (np.isnan(relative_phase_y)).all()):
        mean_x, std_x = [np.nan, np.nan]
        mean_y, std_y = [np.nan, np.nan]
    else:
        print('min, max',np.max(relative_phase_x), np.min(relative_phase_x))
        mean_x, std_x = compute_circular_stat2(relative_phase_x)
        mean_y, std_y = compute_circular_stat2(relative_phase_y)
    return [mean_x, std_x], [mean_y, std_y]

In [14]:
def get_mean_std_curvature(v_,acc_):

    v = np.stack((v_[0], v_[1], v_[2]), axis=-1)
    v_mm_per_s = v*1000
    acc = np.stack((acc_[0], acc_[1], acc_[2]), axis=-1)
    acc_mm_per_s_squared = acc*1000

    cross = np.cross(v_mm_per_s, acc_mm_per_s_squared)
    cross_norm = np.linalg.norm(cross, axis =1)
    norm_v = np.linalg.norm(v_mm_per_s, axis=1)

    curvature = cross_norm/norm_v**3

    return np.nanmean(curvature), np.nanstd(curvature)

In [15]:
def get_features_simplified(subject_id, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec, save =True):
    # just for the task cluster

    NUMBER_OF_SEGMENTS = 8 

    df_metric = pd.DataFrame(columns=['Tool', 'Stitch','Effective_task_duration', 'Idle_time',  'Path_length', 'Jerk', 'Mean_v', 'Std_v', 'Economy_of_volume',
                                       'Bimanual_dexterity','Bimanual_efficacy', 'Mean_relative_phase_x', 'Std_relative_phase_x', 'Mean_relative_phase_y',
                                       'Std_relative_phase_y'])

    list_idle_nh, list_idle_tw = get_list_idle_time(subject_id, i, list_np_segmented_nh_rec, list_np_segmented_tw_rec)
    for s in range (NUMBER_OF_SEGMENTS):
        print(s)
        t_tw = list_np_segmented_tw_rec[s][:,0]
        t_nh = list_np_segmented_nh_rec[s][:,0]

        idle_t_tw = list_idle_tw[s]
        idle_t_nh = list_idle_nh[s]

        path_length_tw = get_path_length_g(t_tw, list_np_segmented_tw_rec[s][:,1:4])
        path_length_nh = get_path_length_g(t_nh, list_np_segmented_nh_rec[s][:,1:4])

        v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(list_np_segmented_tw_rec[s][:,0:4])
        v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(list_np_segmented_nh_rec[s][:,0:4])

        jerk_approximation_tw = get_jerk(list_np_segmented_tw_rec[s][1:,0], v_tw, acc_tw, jerk_tw) #acc not usefull
        jerk_approximation_nh = get_jerk(list_np_segmented_nh_rec[s][1:,0], v_nh, acc_nh, jerk_nh) #acc not usefull

        mean_v_tw, std_v_tw = get_mean_std_v(v_tw)
        mean_v_nh, std_v_nh = get_mean_std_v(v_nh)

        EOV_nh = get_economy_of_volume(list_np_segmented_nh_rec[s][:,1:4], path_length_nh)
        EOV_tw = get_economy_of_volume(list_np_segmented_tw_rec[s][:,1:4], path_length_tw)

        print('BD')
        BD = get_bimanual_dexterity3(subject_id, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
        print('BE')
        BE = get_bimanual_efficiency3(subject_id, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
        print('Relative phase')
        mean_std_relative_phase_x, mean_std_relative_phase_y = get_relative_phase_mean_std(subject_id, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
        
        df_metric.loc[len(df_metric)] = ['TW', s, t_tw.shape[0] * (1.0 / 120.0), idle_t_tw,  path_length_tw, jerk_approximation_tw, mean_v_tw, std_v_tw,
                                          EOV_tw, BD, BE, mean_std_relative_phase_x[0], mean_std_relative_phase_x[1], mean_std_relative_phase_y[0], mean_std_relative_phase_y[1]]
        df_metric.loc[len(df_metric)] = ['NH', s, t_nh.shape[0] * (1.0 / 120.0), idle_t_nh,  path_length_nh, jerk_approximation_nh, mean_v_nh, std_v_nh,
                                          EOV_nh, BD, BE, mean_std_relative_phase_x[0], mean_std_relative_phase_x[1], mean_std_relative_phase_y[0], mean_std_relative_phase_y[1]]

    if save: 
        print('Save')
        directory = f'Features/S_{subject_id}/'
        if not os.path.exists(directory):
            os.makedirs(directory, exist_ok=True)
        df_metric.to_csv(f'{directory}/df_metrics.csv', index=False)
        return df_metric

    else:
        return df_metric
    

In [30]:
def get_features_simplified_array(subject_id, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec, save =True):
    # just for the task cluster

    NUMBER_OF_SEGMENTS = 8 

    data_metrics = np.zeros((NUMBER_OF_SEGMENTS, 13, 2))
    list_idle_nh, list_idle_tw = get_list_idle_time(subject_id, i, list_np_segmented_nh_rec, list_np_segmented_tw_rec)
    for s in range (NUMBER_OF_SEGMENTS):
        print(s)
        t_tw = list_np_segmented_tw_rec[s][:,0]
        t_nh = list_np_segmented_nh_rec[s][:,0]

        idle_t_tw = list_idle_tw[s]
        idle_t_nh = list_idle_nh[s]

        path_length_tw = get_path_length_g(t_tw, list_np_segmented_tw_rec[s][:,1:4])
        path_length_nh = get_path_length_g(t_nh, list_np_segmented_nh_rec[s][:,1:4])

        v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(list_np_segmented_tw_rec[s][:,0:4])
        v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(list_np_segmented_nh_rec[s][:,0:4])

        jerk_approximation_tw = get_jerk(list_np_segmented_tw_rec[s][1:,0], v_tw, acc_tw, jerk_tw) #acc not usefull
        jerk_approximation_nh = get_jerk(list_np_segmented_nh_rec[s][1:,0], v_nh, acc_nh, jerk_nh) #acc not usefull

        mean_v_tw, std_v_tw = get_mean_std_v(v_tw)
        mean_v_nh, std_v_nh = get_mean_std_v(v_nh)

        EOV_nh = get_economy_of_volume(list_np_segmented_nh_rec[s][:,1:4], path_length_nh)
        EOV_tw = get_economy_of_volume(list_np_segmented_tw_rec[s][:,1:4], path_length_tw)
        print('BD')
        BD = get_bimanual_dexterity3(subject_id, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
        print('BE')
        BE = get_bimanual_efficiency3(subject_id, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
        print('Relative phase')
        mean_std_relative_phase_x, mean_std_relative_phase_y = get_relative_phase_mean_std(subject_id, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])

        data_metrics[s, 0, 0] = t_tw.shape[0] * (1.0 / 120.0)
        data_metrics[s, 0, 1] = t_nh.shape[0] * (1.0 / 120.0)
        data_metrics[s, 1, 0] = idle_t_tw
        data_metrics[s, 1, 1] = idle_t_nh
        data_metrics[s, 2, 0] = path_length_tw
        data_metrics[s, 2, 1] = path_length_nh
        data_metrics[s, 3, 0] = jerk_approximation_tw
        data_metrics[s, 3, 1] = jerk_approximation_nh
        data_metrics[s, 4, 0] = mean_v_tw
        data_metrics[s, 4, 1] = mean_v_nh
        data_metrics[s, 5, 0] = std_v_tw
        data_metrics[s, 5, 1] = std_v_nh
        data_metrics[s, 6, 0] = EOV_tw
        data_metrics[s, 6, 1] = EOV_nh
        data_metrics[s, 7, 0] = BD
        data_metrics[s, 7, 1] = BD
        data_metrics[s, 8, 0] = BE
        data_metrics[s, 8, 1] = BE
        data_metrics[s, 9, 0] = mean_std_relative_phase_x[0]
        data_metrics[s, 9, 1] = mean_std_relative_phase_x[0]
        data_metrics[s, 10, 0] = mean_std_relative_phase_x[1]
        data_metrics[s, 10, 1] = mean_std_relative_phase_x[1]
        data_metrics[s, 11, 0] = mean_std_relative_phase_y[0]
        data_metrics[s, 11, 1] = mean_std_relative_phase_y[0]
        data_metrics[s, 12, 0] = mean_std_relative_phase_y[1]
        data_metrics[s, 12, 1] = mean_std_relative_phase_y[1]

    if save: 
        print('Save')
        directory = f'Features/S_{subject_id}/'
        if not os.path.exists(directory):
            os.makedirs(directory, exist_ok=True)
        np.save(f"{directory}/ot_metrics.npy", data_metrics)
        return data_metrics

    else:
        return data_metrics
    

In [16]:
subjects=[1,19, 23,7, 24, 26, 10, 13, 16, 17, 20, 27]
i=0
for subject in subjects:
    needle_holder_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
    tweezers_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')
    delta = tweezers_rec['Time (Seconds)'].values[0] - needle_holder_rec['Time (Seconds)'].values[0]

    selected_nh,selected_tw = run_step1_per_subject(subject,i)

    #apply filter to coordinates
    selected_nh['X.1'] = butter_lowpass_filter(selected_nh['X.1'], cutoff=12)
    selected_nh['Y.1'] = butter_lowpass_filter(selected_nh['Y.1'], cutoff=12)
    selected_nh['Z.1'] = butter_lowpass_filter(selected_nh['Z.1'], cutoff=12)

    selected_tw['X.1'] = butter_lowpass_filter(selected_tw['X.1'], cutoff=12)
    selected_tw['Y.1'] = butter_lowpass_filter(selected_tw['Y.1'], cutoff=12)
    selected_tw['Z.1'] = butter_lowpass_filter(selected_tw['Z.1'], cutoff=12)
    
    selected_tw['Time (Seconds)'] = selected_tw['Time (Seconds)'].values - delta

    #cut the time points in stitches
    dict_segment_time = full_segments_time[i]
    list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw, dict_segment_time)
    list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh, dict_segment_time)

    df_metrics = get_features_simplified(subject, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec, save='True')
    print(subject)
    print(df_metrics)
    i=i+1

0
BD
BE
Relative phase
min, max 1.9861591257458646 -1.844833763912058
1
BD
BE
Relative phase
min, max 1.562943144557169 -1.5190263438921676
2
BD
BE
Relative phase
min, max 1.8151514612738908 -1.604976208927415
3
BD
BE
Relative phase
min, max 1.4810118705070014 -1.3243734607214197
4
BD
BE
Relative phase
min, max 1.5021094589743293 -1.6356295891712436
5
BD
BE
Relative phase
min, max 1.3684554539556915 -1.393407531622844
6
BD
BE
Relative phase
min, max 1.8109848360628489 -1.8317865419373676
7
BD
BE
Relative phase
min, max 1.3150136710370748 -1.8863463177837594
Save
1
   Tool  Stitch  Effective_task_duration   Idle_time  Path_length       Jerk  \
0    TW       0               239.025000   85.283333     8.687535 -31.290140   
1    NH       0               217.241667  107.058333     4.536104 -31.060091   
2    TW       1               135.425000   83.883333     3.845529 -29.650260   
3    NH       1               133.775000   85.525000     2.431008 -29.540011   
4    TW       2              

In [56]:
subjects=[1,19, 23,7, 24, 26, 10, 13, 16, 17, 20, 27]
i=0
for subject in subjects:
    needle_holder_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
    tweezers_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')
    delta = tweezers_rec['Time (Seconds)'].values[0] - needle_holder_rec['Time (Seconds)'].values[0]

    selected_nh,selected_tw = run_step1_per_subject(subject,i)

    #apply filter to coordinates
    selected_nh['X.1'] = butter_lowpass_filter(selected_nh['X.1'], cutoff=12)
    selected_nh['Y.1'] = butter_lowpass_filter(selected_nh['Y.1'], cutoff=12)
    selected_nh['Z.1'] = butter_lowpass_filter(selected_nh['Z.1'], cutoff=12)

    selected_tw['X.1'] = butter_lowpass_filter(selected_tw['X.1'], cutoff=12)
    selected_tw['Y.1'] = butter_lowpass_filter(selected_tw['Y.1'], cutoff=12)
    selected_tw['Z.1'] = butter_lowpass_filter(selected_tw['Z.1'], cutoff=12)
    
    selected_tw['Time (Seconds)'] = selected_tw['Time (Seconds)'].values - delta

    #cut the time points in stitches
    dict_segment_time = full_segments_time[i]
    list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw, dict_segment_time)
    list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh, dict_segment_time)

    df_metrics = get_features_simplified_array(subject, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec)
    print(subject)
    print(df_metrics)
    i=i+1

0
BD
BE
Relative phase
min, max 1.9861591257458646 -1.844833763912058
1
BD
BE
Relative phase
min, max 1.562943144557169 -1.5190263438921676
2
BD
BE
Relative phase
min, max 1.8151514612738908 -1.604976208927415
3
BD
BE
Relative phase
min, max 1.4810118705070014 -1.3243734607214197
4
BD
BE
Relative phase
min, max 1.5021094589743293 -1.6356295891712436
5
BD
BE
Relative phase
min, max 1.3684554539556915 -1.393407531622844
6
BD
BE
Relative phase
min, max 1.8109848360628489 -1.8317865419373676
7
BD
BE
Relative phase
min, max 1.3150136710370748 -1.8863463177837594
Save
1
[[[ 2.39025000e+02  2.17241667e+02]
  [ 8.52833333e+01  1.07058333e+02]
  [ 8.68753499e+00  4.53610438e+00]
  [-3.12901402e+01 -3.10600912e+01]
  [ 3.65424605e-02  2.09103398e-02]
  [ 4.80050663e-02  2.81802085e-02]
  [ 8.98477381e-03  9.06300167e-03]
  [-2.26784996e-02 -2.26784996e-02]
  [ 7.76088459e-01  7.76088459e-01]
  [ 1.80386336e+02  1.80386336e+02]
  [ 2.32395933e+01  2.32395933e+01]
  [ 1.80427735e+02  1.80427735e+0

Test relative phase

In [14]:
def compute_mask_both_tools(subject, i, segment_nh, segment_tw):
    dict_segment_time = full_segments_time[i]
    needle_holder_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
    tweezers_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')
    ref_nh_time = pd_2_numpy_and_segment(needle_holder_rec, dict_segment_time)[0][0,0]
    ref_tw_time = pd_2_numpy_and_segment(tweezers_rec, dict_segment_time)[0][0,0]

    delta_t = ref_nh_time - ref_tw_time
    time_nh = segment_nh[:,0]
    time_tw = segment_tw[:,0]

    mask_time_nh = np.zeros(len(time_nh))
    for index, t in enumerate(time_nh):
        diff = np.abs(t - np.array(time_tw)) <= (delta_t)
        if (np.sum(diff)>0): mask_time_nh[index]=1

    mask_time_tw = np.zeros(len(time_tw))
    for index, t in enumerate(time_tw):
        diff = np.abs(np.array(time_nh) - t) <= delta_t
        if (np.sum(diff)>0): mask_time_tw[index]=1
    
    return mask_time_nh.astype(int), mask_time_tw.astype(int)

In [22]:
subjects=[1]#,19, 23,7, 24, 26, 10, 13, 16, 17, 20, 27]
i=0
for subject in subjects:
    #select data points corresponding to the main task
    selected_nh,selected_tw = run_step1_per_subject(subject,i)

    #apply filter to coordinates
    selected_nh['X.1'] = butter_lowpass_filter(selected_nh['X.1'], cutoff=12)
    selected_nh['Y.1'] = butter_lowpass_filter(selected_nh['Y.1'], cutoff=12)
    selected_nh['Z.1'] = butter_lowpass_filter(selected_nh['Z.1'], cutoff=12)

    selected_tw['X.1'] = butter_lowpass_filter(selected_tw['X.1'], cutoff=12)
    selected_tw['Y.1'] = butter_lowpass_filter(selected_tw['Y.1'], cutoff=12)
    selected_tw['Z.1'] = butter_lowpass_filter(selected_tw['Z.1'], cutoff=12)

    #cut the time points in stitches
    dict_segment_time = full_segments_time[i]
    list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw, dict_segment_time)
    list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh, dict_segment_time)


In [16]:
def get_bimanual_dexterity3(subject, i, segment_nh, segment_tw):

    #v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(segment_nh)
    #v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(segment_tw)
    
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    
    if ((np.sum(mask_nh)<=10) or np.sum(mask_tw)<=10):
        BD=np.nan
    else: 
        nh_points = segment_nh[mask_nh.astype(bool)]
        tw_points = segment_tw[mask_tw.astype(bool)]
        v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(nh_points)
        v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(tw_points)
        
        v_stitch_nh = np.linalg.norm(v_nh, axis=0)
        v_stitch_tw = np.linalg.norm(v_tw, axis=0)
            
        mean_v_tw = np.nanmean(v_stitch_tw)
        mean_v_nh = np.nanmean(v_stitch_nh)

        BD = np.sum((v_stitch_nh - mean_v_nh)*(v_stitch_tw - mean_v_tw))/np.sqrt(np.sum((v_stitch_nh - mean_v_nh)**2)*np.sum((v_stitch_tw - mean_v_tw)**2))
    
    return BD

In [29]:
def get_bimanual_efficiency3(subject, i, segment_nh, segment_tw): #consider only time of selected moments
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    dt = 1/120
    time_nh = segment_nh[mask_nh.astype(bool)][:,0]
    #time_nh = segment_nh[:,0][mask_nh.astype(bool)] #since we have a list we need a boolean mask
    diff_nh = np.diff(time_nh)
    #we don't want to consider the difference of time between 2 points separated by an unselected period
    mask_nh = diff_nh>dt
    diff_nh_ = diff_nh[~mask_nh]
    sum_time_nh = np.sum(diff_nh_)

    time_whole_segment_nh = segment_nh[:,0]
    diff_whole_segment_nh = np.diff(time_whole_segment_nh)
    mask_whole = diff_whole_segment_nh>dt
    diff_whole = diff_whole_segment_nh[~mask_whole]
    sum_time_whole = np.sum(diff_whole)

    if (sum_time_nh ==0): #it is due to imprecision in the algorithm, we should in theory not have 0 values
        BE=np.nan
    else: 
        BE= sum_time_nh/sum_time_whole #sum time is the same for nh and tw

    return BE

In [31]:
for s in range(8):
    print(get_bimanual_efficiency3(subject, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s]))

0.46446671273896617
0.3408432270730111
0.26636895402237665
0.15999145573000106
0.16964359206519056
0.17967032967032967
0.19642857142857142
0.17235123367575095


In [59]:
for s in range (8):
    print(get_bimanual_dexterity3(subject, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s]))

-0.005946783878297836
0.06629251280569184
0.06573593297331476
0.010490837613471856
0.021479702674178005
-0.007092199501124161
0.12721471425235775
0.03980067724269241


In [17]:
def get_relative_phase(subject, i, segment_nh, segment_tw):
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    
    if ((np.sum(mask_nh)<=10) or np.sum(mask_tw)<=10):
        relative_phase_x=np.nan
        relative_phase_y=np.nan
        return relative_phase_x, relative_phase_y
    else: 
        nh_points = segment_nh[mask_nh.astype(bool)]
        tw_points = segment_tw[mask_tw.astype(bool)]


        dt = 1/120
        diff_t_x_y_z_nh = np.diff(nh_points, axis = 0)
        mask_step_bigger_dt_nh = (diff_t_x_y_z_nh[:,0]>dt)
        t_x_y_z_nh = diff_t_x_y_z_nh[~mask_step_bigger_dt_nh]
        v_x_y_z_nh = [t_x_y_z_nh[:,1]/t_x_y_z_nh[:,0], t_x_y_z_nh[:,2]/t_x_y_z_nh[:,0], t_x_y_z_nh[:,3]/t_x_y_z_nh[:,0]]
        x_y_z_nh = nh_points[1:,1:3][~mask_step_bigger_dt_nh]

        diff_t_x_y_z_tw = np.diff(tw_points, axis = 0)
        mask_step_bigger_dt_tw = (diff_t_x_y_z_tw[:,0]>dt)
        t_x_y_z_tw = diff_t_x_y_z_tw[~mask_step_bigger_dt_tw]
        v_x_y_z_tw = [t_x_y_z_tw[:,1]/t_x_y_z_tw[:,0], t_x_y_z_tw[:,2]/t_x_y_z_tw[:,0], t_x_y_z_tw[:,3]/t_x_y_z_tw[:,0]]
        x_y_z_tw = tw_points[1:,1:3][~mask_step_bigger_dt_tw]

        #print(v_x_y_z_nh)
        phi_nh_x = np.arctan(v_x_y_z_nh[0]/x_y_z_nh[:,0])
        phi_tw_x = np.arctan(v_x_y_z_tw[0]/x_y_z_tw[:,0])

        relative_phase_x = phi_nh_x - phi_tw_x

        phi_nh_y = np.arctan(v_x_y_z_nh[1]/x_y_z_nh[:,1])
        phi_tw_y = np.arctan(v_x_y_z_tw[1]/x_y_z_tw[:,1])

        relative_phase_y = phi_nh_y - phi_tw_y

        return relative_phase_x, relative_phase_y


In [23]:
def compute_circular_stat(data):
    return pycircstat.mean(data), pycircstat.std(data)

In [28]:
for s in range(8):
    print(compute_circular_stat(get_relative_phase(subject, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])[0]), compute_circular_stat(get_relative_phase(subject, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])[1]))

(6.281127643441595, 0.41429832735564526) (0.0012677647499118089, 0.6326294396401114)
(0.005524808520276553, 0.29578831323827776) (0.007588221637655853, 0.5620762819882056)
(0.0005006897951789773, 0.20384817185822648) (6.269552760393235, 0.45013548681038884)
(6.27649499584724, 0.22191833779086434) (6.266920732681488, 0.5436033011451784)
(0.0064610438277341, 0.2269603977692718) (6.281455944858819, 0.46869995651308477)
(6.275115437423069, 0.25684579521733514) (6.2333252361221225, 0.6676244100486599)
(6.279424620821119, 0.38810863990634026) (0.02965247730466716, 0.7060883164117122)
(0.011169733398364738, 0.3027048631867433) (6.279437741876621, 0.5881069658370923)
